# (More) In-Depth Python for Engineers

**Welcome!** In the 'Bootcamp' notebook, you learned the basic syntax of Python. Now, we move to the next level: learning how to think like a programmer to solve engineering problems. This lesson focuses on **programming logic**—how to make decisions, repeat tasks, and structure data and functions effectively.

**Objective:** To provide a comprehensive guide to the essential programming structures and techniques used to build engineering models and simulations in Python.

**Learning Goals:**
1.  Master **Control Flow** using `if`/`elif`/`else` statements to make decisions in your code.
2.  Learn to automate repetitive tasks using **`for` loops**.
3.  Gain proficiency with **data structures** (lists and dictionaries) for organizing complex engineering data.
4.  Learn to write more powerful and flexible **functions** that can return multiple values and use default arguments.
5.  Combine all these skills to build a fully interactive simulation of a batch reactor.

## Part 1: Control Flow - Giving Your Code a Brain

Control flow allows your program to execute different pieces of code based on certain conditions. This is how we make our models 'smart'.

### 1.1 `if`, `elif`, `else` - Making Decisions

These statements allow you to run code only if a certain condition is true. This is fundamental for modeling physical boundaries and different behaviors.

**Engineering Example:** Let's determine the flow regime (laminar, transitional, or turbulent) based on the calculated Reynolds number.

In [ ]:
def check_flow_regime(Re):
    """Determines the flow regime based on the Reynolds number."""
    if Re < 2100:
        regime = "Laminar"
    elif Re >= 2100 and Re <= 4000: # 'elif' is 'else if'
        regime = "Transitional"
    else: # 'else' catches all other cases
        regime = "Turbulent"
    return regime

# --- Let's test our function ---
Re_1 = 1500
Re_2 = 3000
Re_3 = 50000

print(f"A Reynolds number of {Re_1} corresponds to {check_flow_regime(Re_1)} flow.")
print(f"A Reynolds number of {Re_2} corresponds to {check_flow_regime(Re_2)} flow.")
print(f"A Reynolds number of {Re_3} corresponds to {check_flow_regime(Re_3)} flow.")

### 1.2 `for` Loops - Automating Repetitive Tasks

A `for` loop is used to iterate over a sequence (like a list) and perform an action for each item. This saves us from writing the same code over and over again.

**Engineering Example:** Let's calculate the vapor pressure of water at several different temperatures using the Antoine equation.

In [ ]:
import numpy as np

def calculate_vapor_pressure(T_celsius):
    """Calculates vapor pressure of water using Antoine equation."""
    # Antoine constants for water (P in mmHg, T in Celsius)
    A, B, C = 8.07131, 1730.63, 233.426
    P_mmHg = 10**(A - (B / (C + T_celsius)))
    return P_mmHg

# A list of temperatures we are interested in
temperatures_C = [20, 40, 60, 80, 100]

# An empty list to store our results
vapor_pressures = []

# The for loop!
# 'for temp in temperatures_C:' means "for each item in the list, temporarily call it 'temp' and do the following:"
for temp in temperatures_C:
    # Calculate the vapor pressure for the current temperature
    P_sat = calculate_vapor_pressure(temp)
    # Add the result to our list
    vapor_pressures.append(P_sat)
    print(f"At {temp} C, the vapor pressure is {P_sat:.2f} mmHg.")

print("\nFinal list of calculated pressures:", vapor_pressures)

## Part 2: Mastering Data Structures

Organizing data effectively is crucial for writing clean and understandable code.

### 2.1 Lists: The Engineer's Go-To Container

We've seen how to create lists and access elements. A common task is building a list dynamically within a loop, as we did above with `vapor_pressures.append(P_sat)`. This is a fundamental pattern for collecting simulation results.

### 2.2 Dictionaries: For Organized, Labeled Data

Dictionaries are perfect for storing structured data with labels, like the physical properties of a chemical. This is much better than having many separate variables (e.g., `k_benzene`, `formula_benzene`, etc.). It keeps related data together.

**Engineering Example:** Store all the parameters for our water vapor pressure calculation in a single dictionary.

In [ ]:
water_data = {
    "name": "Water",
    "formula": "H2O",
    "antoine_coeffs": {"A": 8.07131, "B": 1730.63, "C": 233.426},
    "units": {"T": "Celsius", "P": "mmHg"}
}

# Now we can write a more robust function that takes the dictionary as input
def calculate_P_sat_from_dict(T, chemical_data):
    """Calculates vapor pressure using a chemical data dictionary."""
    A = chemical_data["antoine_coeffs"]["A"]
    B = chemical_data["antoine_coeffs"]["B"]
    C = chemical_data["antoine_coeffs"]["C"]
    
    P_sat = 10**(A - (B / (C + T)))
    return P_sat

# --- Let's test the new function ---
T_boiling = 100.0
P_at_boiling = calculate_P_sat_from_dict(T_boiling, water_data)

print(f"Using the data dictionary for {water_data['name']}...")
print(f"The calculated vapor pressure at {T_boiling} C is {P_at_boiling:.1f} {water_data['units']['P']}.")
print("(This is atmospheric pressure, as expected!)")

## Part 3: Writing Powerful and Reusable Functions

Well-designed functions are the key to building complex simulations without creating a mess.

### 3.1 Functions That Return Multiple Values
Our engineering models often need to output multiple results. For example, a reactor model needs to tell us the final concentration of both reactants and products. Functions can easily return multiple values (as a `tuple`).

In [ ]:
def batch_reaction_step(C_A_initial, C_B_initial, k, dt):
    """Calculates concentrations after one time step for A -> B."""
    # Rate is based on the initial concentration for this small time step
    rate = k * C_A_initial
    
    # Amount of A reacted in this time step
    A_reacted = rate * dt
    
    # New concentrations
    C_A_final = C_A_initial - A_reacted
    C_B_final = C_B_initial + A_reacted
    
    return C_A_final, C_B_final # Returning two values!

# --- Let's test it ---
C_A_start, C_B_start = 1.0, 0.0

# We can "unpack" the returned tuple into multiple variables
C_A_next, C_B_next = batch_reaction_step(C_A_start, C_B_start, k=0.1, dt=1.0)

print(f"Initial concentrations: C_A={C_A_start:.2f}, C_B={C_B_start:.2f}")
print(f"After one time step:   C_A={C_A_next:.2f}, C_B={C_B_next:.2f}")

## Part 4: Interactive Capstone - A Batch Reactor Simulation

Let's put everything together! We will build a complete simulation of a batch reactor using all the concepts we've learned: functions, `for` loops, `if` statements, and lists. We will then make it interactive to see the effect of our parameters in real time.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def simulate_batch_reactor(k=0.1, C_A0=1.0, total_time=30.0):
    """Runs a full batch reactor simulation and plots the results."""
    
    # --- Simulation Setup ---
    dt = 0.1 # Time step size (minutes)
    num_steps = int(total_time / dt)
    
    # Use lists to store the history of our concentrations
    time_history = [0]
    C_A_history = [C_A0]
    C_B_history = [0.0]
    
    # Set initial values for the loop
    C_A_current = C_A0
    C_B_current = 0.0
    
    # --- The Main Simulation Loop ---
    for i in range(num_steps):
        # Use an 'if' statement to prevent negative concentrations
        if C_A_current < 0:
            C_A_current = 0
        
        # Calculate concentrations for the next time step
        C_A_current, C_B_current = batch_reaction_step(C_A_current, C_B_current, k, dt)
        
        # Store the results
        C_A_history.append(C_A_current)
        C_B_history.append(C_B_current)
        time_history.append((i + 1) * dt)
    
    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(time_history, C_A_history, label='Concentration of A (reactant)')
    ax.plot(time_history, C_B_history, label='Concentration of B (product)')
    ax.set_xlabel("Time (minutes)")
    ax.set_ylabel("Concentration (mol/L)")
    ax.set_title("Batch Reactor Concentration Profile")
    ax.legend()
    ax.grid(True)
    ax.set_ylim(bottom=0)
    plt.show()

# Create interactive widgets and link them to our simulation function
interactive_plot = widgets.interactive(simulate_batch_reactor,
                                       k=widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Rate Constant, k:'),
                                       C_A0=widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Initial C_A:'),
                                       total_time=widgets.FloatSlider(value=30, min=10, max=100, step=5, description='Total Time:'))

# Display the interactive dashboard
display(interactive_plot)

## Conclusion: You are a Programmer! 

Congratulations! You have now mastered the essential logical structures of programming that are required for engineering simulation.

You have learned:
✅ How to use **`if/elif/else`** to make your code respond to different conditions.

✅ How to use **`for` loops** to automate calculations over a range of data or time.

✅ How to effectively use **lists and dictionaries** to organize the data for your models.

✅ How to write **powerful functions** that form the reusable heart of any good simulation.

With these skills, you are now more well-prepared to understand and build the more complex models in the subsequent lessons on reaction engineering, thermodynamics, separations, and process control.